# Import libraries and dataset

In [1]:
import pandas as pd
import numpy as np 
import os 

path = r'C:\Users\ctede\OneDrive\Desktop\Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_new_columns.pkl'))

In [2]:
#Preview dataframe
ords_prods_merge.head()

,order_id,customer_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,new_busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,0.0,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders


In [3]:
#How many rows does the dataframe have? 
ords_prods_merge.shape

(32404859, 18)

### 2 In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
#Using agg() and groupby() on the entire dataframe ords_prods_merge
ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### 3 Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

#### Performing the agg() function on the entire dataframe includes all of the department ids. In the subset dataframe only a handful of department ids (8 of them) were included in the output because the subset was only composed of the first 1 million rows. When comparing the order number means for the department ids that were in both outputs, the order number mean in the entire dataframe output is smaller than in the subset dataframe. For example, in the subset, department id 4 had an order number mean of 18.825780 but in the entire dataframe, department id 4 has a mean of 17.811403. But department id 17 had an order number mean of 11.294069 in the subset, and 15.694469 in the entire dataframe. I think bc the entire dataframe has more data, the order number means from the entire dataframe are more accurate than the subset. 

### 4 Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [5]:
#Use transform() to create a max_order column to show the maximum number of orders per customer
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['customer_id'])['order_number'].transform(np.max)

In [6]:
#Use loc() functions to create criteria
#Loyal customers > 40 max orders
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
#Regular customers <=40 and >10 max orders
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
#New customers <= 10 max orders
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
#Show frequency of each loyalty value/status
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [10]:
#View loyalty flag column and customer id 
ords_prods_merge[['customer_id', 'loyalty_flag','order_number']].head(15)

,customer_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


### 5 The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [11]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean','max','min','median']})

prices                     
                       mean      max  min median
loyalty_flag                                    
Loyal customer    10.386336  99999.0  1.0    7.4
New customer      13.294670  99999.0  1.0    7.4
Regular customer  12.495717  99999.0  1.0    7.4

#### New customers on average purchase products that are more expensive (13.3 dollars) vs. loyal customers that purchase on average products that are 10.3 dollars. The regular customers are in between the new and loyal customers. 

### 6 The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

In [12]:
pd.set_option('display.max_columns', 50)

In [20]:
#Transform and create a new column: avg_order_price
ords_prods_merge['avg_order_price'] = ords_prods_merge.groupby(['customer_id'])['prices'].transform(np.average)

In [21]:
#loc () function
#If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
ords_prods_merge.loc[ords_prods_merge['avg_order_price'] < 10, 'spending_flag'] = 'Low spender'

In [22]:
#loc () function 
#If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”
ords_prods_merge.loc[ords_prods_merge['avg_order_price'] >= 10, 'spending_flag'] = 'High spender'

In [23]:
#Show frequency of spending flag
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770646
High spender      634213
Name: spending_flag, dtype: int64

In [24]:
ords_prods_merge.head()

,order_id,customer_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,new_busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_price,spending_flag
0,2539329,1,1,2,8,0.0,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender


### 7 In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

In [25]:
#Transform and create a new column: order_frequency
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['customer_id'])['days_since_prior_order'].transform(np.median)

In [26]:
#If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'order_freq_flag'] = 'Non-frequent customer'

In [29]:
#If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'order_freq_flag'] = 'Regular customer'

In [30]:
#If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'order_freq_flag'] = 'Freqeunt customer'

In [31]:
#Check flags
ords_prods_merge[['customer_id', 'order_freq_flag','days_since_prior_order']].head(15)

,customer_id,order_freq_flag,days_since_prior_order
0,1,Regular customer,0.0
1,1,Regular customer,15.0
2,1,Regular customer,21.0
3,1,Regular customer,29.0
4,1,Regular customer,28.0
5,1,Regular customer,19.0
6,1,Regular customer,20.0
7,1,Regular customer,14.0
8,1,Regular customer,0.0
9,1,Regular customer,30.0


In [32]:
#Show frequency/counts of order frequency
ords_prods_merge['order_freq_flag'].value_counts()

Freqeunt customer        22796659
Regular customer          6921472
Non-frequent customer     2686728
Name: order_freq_flag, dtype: int64

#### There are more frequent customers (customers ordering more often because there are less days between each order [<10]).  

# Export a COPY of ords_prods_merge with new columns

In [33]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'COPY_orders_products_new_columns.pkl'))